In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
### Установим красивые дефолтные настройки
### Может быть лень постоянно прописывать
### У графиков параметры цвета, размера, шрифта
### Можно положить их в словарь дефолтных настроек

import matplotlib as mlp

mlp.rcParams['lines.linewidth'] = 5

mlp.rcParams['xtick.major.size'] = 20
mlp.rcParams['xtick.major.width'] = 5
mlp.rcParams['xtick.labelsize'] = 20
mlp.rcParams['xtick.color'] = '#FF5533'

mlp.rcParams['ytick.major.size'] = 20
mlp.rcParams['ytick.major.width'] = 5
mlp.rcParams['ytick.labelsize'] = 20
mlp.rcParams['ytick.color'] = '#FF5533'

mlp.rcParams['axes.labelsize'] = 20
mlp.rcParams['axes.titlesize'] = 20
mlp.rcParams['axes.titlecolor'] = '#00B050'
mlp.rcParams['axes.labelcolor'] = '#00B050'

### KNN: часть 1

Сгенерируем случайную выборку и попробуем пронаблюдать, как при разных значениях `K` будут меняться результаты модели, основанной на ближайших соседях!


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.display.max_columns = 500

In [ ]:
X = np.random.uniform(low=0, high=20, size=(500, 1))

def target(x):  
    return x + 3 * np.sin(2*x)


noise = np.random.normal(loc=0, scale=1, size=(500,1))

Y = target(X) + noise

In [ ]:
import seaborn as sns

fig = plt.figure()

fig.set_size_inches(16, 10)

sns.scatterplot(X.ravel(), Y.ravel())

plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

splitter = KFold(n_splits=5, shuffle=True, random_state=33)

linear = LinearRegression()
knn = KNeighborsRegressor(n_neighbors=3) 

linear_cv = cross_validate(linear, X, Y, 
                           cv=splitter, scoring='neg_mean_squared_error',
                           return_train_score=True)

knn_cv = cross_validate(knn, X, Y, 
                        cv=splitter, scoring='neg_mean_squared_error',
                        return_train_score=True)

In [ ]:
print(f"Среднее MSE линейной модели на трейне: {-np.mean(linear_cv['train_score']).round(3)}")
print(f"Среднее MSE линейной модели на тесте: {-np.mean(linear_cv['test_score']).round(3)}")

print(f"\nСреднее MSE KNN модели на трейне: {-np.mean(knn_cv['train_score']).round(3)}")
print(f"Среднее MSE KNN модели на тесте: {-np.mean(knn_cv['test_score']).round(3)}")

In [ ]:
### Обучим модель на всей выборке

linear.fit(X, Y)
knn.fit(X, Y)

In [ ]:
### Пришли новые данные!

new_X = np.random.uniform(low=20, high=30, size=(50, 1))

new_noise = np.random.normal(loc=0, scale=1, size=(50, 1))

new_Y = target(new_X) + new_noise

knn_preds = knn.predict(new_X)
linear_preds = linear.predict(new_X)

In [ ]:
print(f"MSE KNN модели на новых данных: {np.mean((knn_preds-new_Y)**2).round(3)}")
print(f"MSE линейной модели новых данных: {np.mean((linear_preds-new_Y)**2).round(3)}")

In [ ]:
fig = plt.figure()

fig.set_size_inches(16, 10)

sns.scatterplot(new_X.ravel(), new_Y.ravel())
sns.scatterplot(X.ravel(), Y.ravel())
sns.scatterplot(new_X.ravel(), linear_preds.ravel())
sns.scatterplot(new_X.ravel(), knn_preds.ravel())


plt.legend(["new_data", "training_data", "linear_preds", "knn_preds"])

plt.show()

### KNN: подбор числа K

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    test_size=0.33, 
                                                    random_state=33)

fig = plt.figure()
fig.set_size_inches(20, 20)

for K in range(1, 10):

    knn = KNeighborsRegressor(n_neighbors=K) 
    knn.fit(X_train, y_train)
    
    preds_test = knn.predict(X_test)
    
    ax = fig.add_subplot(3, 3, K)
    
    sns.scatterplot(X.ravel(), Y.ravel())
    sns.lineplot(X_test.ravel(), preds_test.ravel(), size=3, color='#FF5533')
    plt.legend(["all_data", f"knn_with_{K}_neighbors"])

fig.tight_layout()

plt.show()

Совсем большие K тоже плохо, ведь чем оно больше, тем прогноз больше похож просто на среднее по трейну.

In [ ]:
X_train.shape

In [ ]:
knn = KNeighborsRegressor(n_neighbors=335)

fig = plt.figure()
fig.set_size_inches(16, 10)

knn.fit(X_train, y_train)
    
preds_test = knn.predict(X_test)

sns.scatterplot(X.ravel(), Y.ravel())
sns.lineplot(X_test.ravel(), preds_test.ravel(), size=3, color='#FF5533')
plt.legend(["all_data", f"knn_with_100_neighbors"])

plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}

search = GridSearchCV(knn, param_grid, cv=splitter)

search.fit(X, Y)

print(f"Best parameter (CV score={search.best_score_:.5f}):")
print(search.best_params_)

Ширина Гауссовского ядра

$$
w_i = \frac{1}{\sqrt{2\cdot\pi}} \cdot exp(-\frac{1}{2}\cdot \frac{\rho^2 (x, x_i)}{h^2})
$$

In [ ]:
from sklearn.neighbors import KernelDensity

fig = plt.figure()
fig.set_size_inches(20, 40)

k=1

for h in [0.01, 0.01, 10, 100, 500]:
    
    def kernel(distances, h=h):
    
        const = 1 / (np.sqrt(2 * np.pi))
        power = (-1/2) * ((distances)**2) / h**2
    
        return const * np.exp(power)

    knn = KNeighborsRegressor(n_neighbors=8, weights=kernel)
    knn.fit(X_train, y_train)
    
    preds_test = knn.predict(X_test)
    
    ax = fig.add_subplot(5, 1, k)
    
    k += 1
    
    sns.scatterplot(X.ravel(), Y.ravel())
    sns.lineplot(X_test.ravel(), preds_test.ravel(), size=3, color='#FF5533')
    plt.legend(["all_data", f"knn_with_{h}-windowed_kernel_neighbors"])

fig.tight_layout()


### KNN: сегментация клиентов

In [ ]:
data = pd.read_excel('processed_segmentation.xlsx')

data.head()

In [ ]:
X = data.drop("Segmentation", axis=1)
Y = data["Segmentation"]

One vs Rest без подбора гиперпараметров:

<div>
<img src="onevsrest.png" width="1000"/>
</div>

In [ ]:
def kernel(distances, h=1):
    
        const = 1 / (np.sqrt(2 * np.pi))
        power = (-1/2) * ((distances)**2) / h**2
    
        return const * np.exp(power)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    random_state=0,
                                                    test_size=0.2)

pipe = Pipeline([('scaler', StandardScaler()), 
                 ('KNN', KNeighborsClassifier(weights=kernel))])

pipe.fit(X_train, Y_train)

In [ ]:
### Замерим accuracy

print(f"Качество на трейне: {np.mean(pipe.predict(X_train) == Y_train).round(3)}")

print(f"Качество на тесте: {np.mean(pipe.predict(X_test) == Y_test).round(3)}")

In [ ]:
pipe.get_params()

In [ ]:
np.linspace(1, 5, 5, dtype=int)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'KNN__n_neighbors': np.linspace(1, 100, 10, dtype=int),
    'KNN__p': np.linspace(1, 5, 5, dtype=int),
    'KNN__weights': ['uniform', 'distance', kernel]
}

from sklearn.model_selection import PredefinedSplit

test_fold = [0 if x in X_train.index else -1 for x in X.index]
ps = PredefinedSplit(test_fold)

search = GridSearchCV(pipe, param_grid, cv=ps)

search.fit(X, Y)

print(f"Best parameter (CV score={search.best_score_:.5f}):")
print(search.best_params_)